# Primary Research Weigthing  Use Case

- Whenever you find yourself conducting a primary research htere is always the question whether your sample is representative of the whole population

- One of the strategies that can be used for a survey to be representative is to stratify the sample into  subsets that match the population as a whole. When conducting a massive primary research, you frequently  want your sample to follow demographic distributions of the original population, oftentimes available as public data

- But pannels are not perfect and it is likely you won't find a perfect match in each subset of your data, therefore you need to perform what we call **sample weighting**, which means in simple terms giving different weights for each response so the aggregate sum of response weights is approximately the whole population distribution

- You can model this problem as a linear programming problem where you want to minimize the aggregate error of the distribution in each demographic class vs the population itself

- The problem is subject to constraints

## Whole population data

In [1]:
target_proportion_gender = {
    'Homem' : 0.5,
    'Mulher': 0.5,
    'Prefiro não responder': 0.0, 
    'Outro': 0.0,
}

In [2]:
target_proportion_income_level = {
    'Acima de R$ 24.241': 0.028,
    'Entre R$ 12.121 e R$24.240': 0.132, 
    'Entre R$ 4.849 e R$12.120': 0.333, 
    'Até R$4.848': 0.507,
}

In [3]:
target_proportion_age_group = {
    '16-19': 0.04031725, 
    '20-24': 0.100462657,
    '25-29': 0.096497026, 
    '30-39': 0.208856576, 
    '40-49': 0.182419035,  
    '50-59': 0.163912756, 
    '60-64': 0.064771976,
    '65+'  : 0.142762723,
}

In [4]:
target_proportion_region = {
    'Norte': 0.089,
    'Nordeste': 0.27,
    'Centro_Oeste': 0.078,
    'Sudeste': 0.42,
    'Sul': 0.143,
}

## Simpler use case - weigthing with proportion to only one variable (e.g. age)

In [5]:
# load libraries to handle data and linear programming
import pandas as pd
import numpy as np
import pulp as lp

In [6]:
df_responses = pd.read_csv("./survey_responses.csv")

C:\Users\gransotto gustavo\AppData\Local\Temp\ipykernel_4168\667192333.py:1: DtypeWarning: Columns (62,63,64,65,66,67,578,584,585,631,637) have mixed types. Specify dtype option on import or set low_memory=False.
  df_responses = pd.read_csv("./survey_responses.csv")


In [13]:
df_responses.columns[:30]

Index(['Respondent', 'Qualified', 'Term_reason', 'Start_time_GMT',
       'End_time_GMT', 'Panel', 'age_group', 'metropolitan_area', 'region',
       'region_metropolitan_area', 'watches_soccer',
       'brazilian_championships_interested',
       'interested_in_campeonato_brasileiro_a',
       'interested_in_campeonato_brasileiro_b',
       'watches_campeonato_brasileiro', 'watches_campeonato_brasileiro_a',
       'watches_campeonato_brasileiro_b', 'watched_championships',
       'full_launch', 'qc_speeder', 'qc_attention_sky_color', 'Q1', 'Q2', 'Q3',
       'Q4', 'Q5', 'Q6', 'Q7.Futebol_Brasileiro_Masculino',
       'Q7.Futebol_Brasileiro_Feminino', 'Q7.Futebol_Europeu'],
      dtype='object')

In [22]:
col_subset = ["Respondent","Qualified","age_group"]
col_subset += ["Q"+str(i) for i in range(1,6)]
col_subset

['Respondent', 'Qualified', 'age_group', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5']

In [31]:
df = (df_responses[col_subset].copy()
      .rename(columns={"Respondent":"id", "Qualified": "is_valid", "Q1":"age", 
                       "Q2":"state", "Q3":"city", "Q4": "income", "Q5":"gender"}))
df

,id,is_valid,age_group,age,state,city,income,gender
0,231586344,1,40-49,45,Paraná,Ponta Grossa,Entre R$ 12.121 e R$24.240,Homem
1,1496882695,1,50-59,50,Pernambuco,Caruaru,Até R$4.848,Homem
2,102553465,1,50-59,50,Minas Gerais,Belo Horizonte,Entre R$ 4.849 e R$12.120,Mulher
3,185896109,1,30-39,32,São Paulo,São Paulo,Entre R$ 4.849 e R$12.120,Mulher
4,1511021362,1,30-39,34,Paraíba,Campina Grande,Entre R$ 12.121 e R$24.240,Mulher
...,...,...,...,...,...,...,...,...
1898,1952468339,1,20-24,24,Paraná,Arapongas,Até R$4.848,Homem
1899,1626730749,1,60-64,63,Distrito Federal,Brasília,Entre R$ 12.121 e R$24.240,Homem
1900,236065267,1,50-59,53,Espírito Santo,São Mateus,Entre R$ 4.849 e R$12.120,Homem
1901,158055050,1,20-24,24,São Paulo,Sorocaba,Entre R$ 4.849 e R$12.120,Homem


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1903 entries, 0 to 1902
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1903 non-null   int64 
 1   is_valid   1903 non-null   int64 
 2   age_group  1903 non-null   object
 3   age        1903 non-null   int64 
 4   state      1903 non-null   object
 5   city       1903 non-null   object
 6   income     1903 non-null   object
 7   gender     1903 non-null   object
dtypes: int64(3), object(5)
memory usage: 119.1+ KB


In [33]:
df.apply(pd.Series.nunique)

id           1903
is_valid        1
age_group       8
age            64
state          27
city          549
income          4
gender          4
dtype: int64

In [34]:
df = df.drop("is_valid", axis=1)

In [37]:
responses_age_count = df["age_group"].value_counts().to_dict()
responses_age_count

{'30-39': 425,
 '40-49': 386,
 '25-29': 289,
 '50-59': 283,
 '20-24': 239,
 '60-64': 121,
 '65+': 91,
 '16-19': 69}

In [92]:
total_respondents = 1903

In [108]:
responses_age_perc = (df["age_group"].value_counts() / total_respondents).to_dict()
responses_age_perc

{'30-39': 0.2233315817130846,
 '40-49': 0.20283762480294273,
 '25-29': 0.15186547556489752,
 '50-59': 0.1487125591171834,
 '20-24': 0.1255911718339464,
 '60-64': 0.06358381502890173,
 '65+': 0.047819232790331054,
 '16-19': 0.03625853914871256}

In [123]:
age_weights = dict([(age,target_proportion_age_group[age]/responses_age_perc[age]) for age in target_proportion_age_group])
age_weights

{'16-19': 1.1119380688405796,
 '20-24': 0.7999181433933054,
 '25-29': 0.6354112127266437,
 '30-39': 0.9351860332423529,
 '40-49': 0.8993352943134715,
 '50-59': 1.102211924621908,
 '60-64': 1.0186865316363636,
 '65+': 2.9854666139450554}

In [124]:
df["weights"] = df["age_group"].apply(lambda age: age_weights[age])
df["weights"].sum()

1902.999998097

### Problem definition
- Assign weights to each respondent (~1900 variables)
- Weights should be constrained between 0.5 and 1.5
- Each age group represents a constraint, the sum of weights for that specific category subtracted from the actual proportion in population should be lower than a specific threshold
- The objective is to minimize the sum of errors in all age-groups 

In [113]:
vars_dict = lp.LpVariable.dicts(name="w",indices=list(df["id"].values.astype(str)),lowBound=0.5, upBound=3.0, cat='Continuous')

In [114]:
target_age_count = dict([(age, target_proportion_age_group[age] * total_respondents) for age in target_proportion_age_group])
target_age_count

{'16-19': 76.72372675,
 '20-24': 191.180436271,
 '25-29': 183.633840478,
 '30-39': 397.45406412799997,
 '40-49': 347.143423605,
 '50-59': 311.925974668,
 '60-64': 123.26107032799999,
 '65+': 271.677461869}

In [115]:
threshold = 0.1

In [116]:
model = lp.LpProblem("minimize_errors",lp.LpMinimize)

In [117]:
# Objective function
model += lp.lpSum([vars_dict[var]*1.0 for var in vars_dict]) - total_respondents

In [118]:
# Constraints
for age in target_age_count:
    age_count = lp.lpSum([vars_dict[str(var_id)] for var_id in df.loc[df["age_group"] == age,"id"].values])
    model += age_count - target_age_count[age] <= threshold
    model += age_count - target_age_count[age] >= -threshold

In [119]:
model.solve()

1

In [120]:
lp.LpStatus[model.status]

'Optimal'

In [121]:
df["weigths"] = df["id"].apply(lambda i: vars_dict[str(i)].value())

In [122]:
df["weigths"].value_counts()

0.500000    1519
3.000000     376
2.623727       1
0.825975       1
1.577462       1
2.854064       1
2.033841       1
2.043424       1
0.661070       1
2.080436       1
Name: weigths, dtype: int64